FILTRAR CNPJS INATIVOS
ADICIONAR DSC LOGRADOURO NA CONSULTA


In [38]:
import requests

url = "https://www.gov.br/saude/pt-br/composicao/sectics/farmacia-popular/arquivos/farmacias_credenciadas_pfpb_atualizada.xlsx/@@download/file"
file_path = r'C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto BD_EngSoft\PPGTI_2024-1\Farmacias_Credenciadas.xlsx'

response = requests.get(url)
if response.status_code == 200:
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully and saved to {file_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


File downloaded successfully and saved to C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto BD_EngSoft\PPGTI_2024-1\Farmacias_Credenciadas.xlsx


In [39]:
import requests

url = "https://www.gov.br/saude/pt-br/composicao/sectics/farmacia-popular/arquivos/elenco-de-medicamentos-e-insumos-pfpb-02-2024"
file_path = r'C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto BD_EngSoft\PPGTI_2024-1\Medicamentos_Credenciadas.pdf'

response = requests.get(url)
if response.status_code == 200:
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully and saved to {file_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")





File downloaded successfully and saved to C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto BD_EngSoft\PPGTI_2024-1\Medicamentos_Credenciadas.pdf


In [44]:
import fitz  # PyMuPDF
import pandas as pd
import re

# Função para extrair texto de todas as páginas do PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Função para filtrar linhas que contêm letras minúsculas
def filter_text(text):
    # Dividir o texto em linhas
    lines = text.split('\n')
    
    # Filtrar linhas que contêm letras minúsculas
    filtered_lines = [line for line in lines if not re.search('[A-Z]', line)]
    
    return filtered_lines

# Função para processar o texto extraído em um dataframe
def process_text_to_dataframe(lines):
    if not lines:
        return pd.DataFrame()

    # Supondo que a primeira linha contém os cabeçalhos das colunas
    headers = re.split(r'\s{2,}', lines[0].strip())
    
    # Processar linhas subsequentes
    data = []
    for line in lines[1:]:
        # Dividir a linha em colunas com base em espaços múltiplos
        columns = re.split(r'\s{2,}', line.strip())
        if len(columns) == len(headers):  # Certificar-se de que o número de colunas é consistente
            data.append(columns)
    
    # Criar o DataFrame
    df = pd.DataFrame(data, columns=headers)
    
    # Remover linhas vazias
    df.dropna(how='all', inplace=True)
    
    # Remover duplicatas
    df.drop_duplicates(inplace=True)
    
    return df

# Caminho para o arquivo PDF
pdf_path = 'Medicamentos_Credenciadas.pdf'

# Extrair texto do PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Filtrar o texto para remover linhas com letras minúsculas
filtered_text = filter_text(pdf_text)

# Processar o texto filtrado para criar o DataFrame
df = process_text_to_dataframe(filtered_text)

# Adicionar um cabeçalho personalizado
df.columns = ['Medicamentos']

# Capitalizar a primeira letra de cada linha
df['Medicamentos'] = df['Medicamentos'].str.capitalize()

# Exibir o DataFrame
print(df)

# Salvar o DataFrame em um arquivo CSV
df.to_csv('Medicamentos.csv', index=False)


                                         Medicamentos
0                        Brometo de ipratrópio 0,25mg
1                Dipropionato de beclometasona 200mcg
2                Dipropionato de beclometasona 250mcg
3                 Dipropionato de beclometasona 50mcg
4                        Sulfato de salbutamol 100mcg
5                           Sulfato de salbutamol 5mg
6                      Cloridrato de metformina 500mg
7    Cloridrato de metformina 500mg - ação prolongada
8                      Cloridrato de metformina 850mg
9                                   Glibenclamida 5mg
10                   Insulina humana regular 100ui/ml
11                           Insulina humana 100ui/ml
12                                      Atenolol 25mg
13                        Besilato de anlodipino 5 mg
14                                     Captopril 25mg
15                     Cloridrato de propranolol 40mg
16                             Hidroclorotiazida 25mg
17                          

In [41]:
import pandas as pd
import requests
from geopy.geocoders import ArcGIS
import time

geolocator = ArcGIS()

def consultar_cnpj_brasilio(cnpj):
    url = f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        info = {
            "CNPJ": data.get("cnpj", ""),
            "Nome": data.get("razao_social", ""),
            "Fantasia": data.get("nome_fantasia", ""),
            "Tipo_Logradouro": data.get("descricao_tipo_de_logradouro", ""),
            "Logradouro": data.get("logradouro", ""),
            "Número": data.get("numero", ""),
            "Município": data.get("municipio", ""),
            "Bairro": data.get("bairro", ""),
            "UF": data.get("uf", ""),
            "Telefone": data.get("telefone", ""),
            "Situação": data.get("descricao_situacao_cadastral", "")
        }
        return info
    return None

def get_coordinates(tipo_logradouro, logradouro, numero, bairro, cidade, estado):
    location = geolocator.geocode(f"{tipo_logradouro} +   + {logradouro}, {numero}, {bairro} ,{cidade}, {estado}")
    if location:
        return location.latitude, location.longitude
    return None

# Ler o arquivo .xlsx e coletar os dados da coluna CNPJ
df = pd.read_excel(file_path, skiprows=12, usecols="B:H")
# df = df[(df['UF'] == "AC")] # Possível filtro de estafos do banco

cnpjs = df['CNPJ']

# Tratar os CNPJs removendo ".", "/" e "-"
cnpjs = cnpjs.str.replace(".", "").str.replace("/", "").str.replace("-", "")

# Processar cada CNPJ e obter informações da Brasil.io com timer
final_data = []
for cnpj in cnpjs:
    cnpj_info = consultar_cnpj_brasilio(cnpj)
    if cnpj_info:
        tipo_logradouro = cnpj_info.get("Tipo_Logradouro", "")
        logradouro = cnpj_info.get("Logradouro", "")
        numero = cnpj_info.get("Número", "")
        bairro = cnpj_info.get("Bairro", "")
        cidade = cnpj_info.get("Município", "")
        estado = cnpj_info.get("UF", "")
        situacao = cnpj_info.get("descricao_situacao_cadastral", "")

        coordinates = get_coordinates(tipo_logradouro, logradouro, numero, bairro, cidade, estado)
        if coordinates:
            cnpj_info["Latitude"] = coordinates[0]
            cnpj_info["Longitude"] = coordinates[1]
            final_data.append(cnpj_info)
        else:
            print("Coordenadas não encontradas para o endereço fornecido.")
    else:
        print(f"Não foi possível obter informações para o CNPJ {cnpj}")
    time.sleep(0.06)  # Esperar um tempo entre consultas

# Criar DataFrame final e salva    r em um arquivo .csv
final_df = pd.DataFrame(final_data)
final_df.to_csv('dados_finais.csv', index=False)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
import os
import pandas as pd

# Diretório contendo os arquivos CSV
diretorio = r'C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto BD_EngSoft\PPGTI_2024-1\dados_estado'

# Lista para armazenar os DataFrames de cada arquivo CSV
frames = []

# Percorrer todos os arquivos na pasta
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(diretorio, arquivo)
        # Lê o arquivo CSV e adiciona ao DataFrame
        df = pd.read_csv(caminho_arquivo, dtype=str)
        frames.append(df)

# Concatena todos os DataFrames em um único DataFrame
df_final = pd.concat(frames, ignore_index=True)

# Exibe o DataFrame resultante
print(df_final)

# Se desejar, você pode salvar o DataFrame unificado em um novo arquivo CSV
df_final.to_csv('resultado.csv', index=False)


In [ ]:
import csv
import json

def csv_to_json(csv_file):
    json_data = []

    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row['Situação'] == 'ATIVA':
                json_row = {
                    "nome": row['Nome'],
                    "longLat": [float(row['Longitude']), float(row['Latitude'])],
                    "nomeFantasia": row['Fantasia'],
                    "endereco": {
                        "rua": row['Tipo_Logradouro'] + ' ' + row['Logradouro'],
                        "numero": row['Número'],
                        "bairro": row['Bairro'],
                        "municipio": row['Município'],
                        "estado": row['UF']
                    }
                }
                json_data.append(json_row)

    return json_data

def main():
    csv_file = "resultado.csv"  
    json_data = csv_to_json(csv_file)
    with open("dados.json", 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    main()
